In [1]:
import os
import torch
import polars as pl
import pandas as pd
import numpy as np
import seaborn as sns
import warnings
from feature_eng.scalers import ranged_scaler
from datetime import datetime, timedelta
#from mpge.rca import mpge_root_cause_diagnosis
warnings.filterwarnings("ignore", category=UserWarning) 
warnings.simplefilter(action='ignore', category=FutureWarning)
pd.options.mode.chained_assignment = None

import warnings

def fxn():
    warnings.warn("deprecated", DeprecationWarning)

with warnings.catch_warnings():
    warnings.simplefilter("ignore")
    fxn()

# Or if you are using > Python 3.11:
with warnings.catch_warnings(action="ignore"):
    fxn()

In [2]:
os.environ["CUBLAS_WORKSPACE_CONFIG"] = ":4096:8"
os.environ["CUDA_LAUNCH_BLOCKING"] = "1"


In [3]:
from tkgngc.embeddings import PretrainedTKGEmbeddingWithTimestamps
from tkgngc.data_processing import TKGNGCDataProcessor
from tkgngc.model import train_model

In [4]:
cats_df = pl.read_csv("data/data.csv", separator=",")  
metadata = pl.read_csv('data/metadata.csv',separator=',')
potential_causes = metadata['root_cause'].unique().to_list()

for col in cats_df.columns:
    unique_vals = cats_df[col].n_unique()
    data_type = cats_df[col].dtype
    bad_dtypes = [pl.Date,pl.Datetime,pl.Utf8]
    if ((unique_vals >= 50) & (data_type not in bad_dtypes) ):
        cats_df = cats_df.with_columns(ranged_scaler(cats_df[col]))
    else:
        continue

In [5]:
cats_df = cats_df.with_columns(
    pl.col('timestamp').str.to_datetime("%Y-%m-%d %H:%M:%S"),
    pl.Series("entity_id",range(len(cats_df)))
)

In [6]:
cats_rows_list = metadata.rows(named=True)
cats_df.head()

timestamp,aimp,amud,arnd,asin1,asin2,adbr,adfl,bed1,bed2,bfo1,bfo2,bso1,bso2,bso3,ced1,cfo1,cso1,y,category,entity_id
datetime[μs],f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,i64
2023-01-01 00:00:00,0.0,0.571429,0.25,0.5,0.5,0.0,0.0,0.33599,0.315382,0.130919,0.116409,0.409726,0.246024,0.14197,0.112819,0.550194,0.406688,0.0,0.0,0
2023-01-01 00:00:01,0.0,0.571429,0.252001,0.50001,0.5001,0.0,0.0,0.33599,0.315382,0.130919,0.116409,0.40973,0.246024,0.14197,0.112819,0.550194,0.406691,0.0,0.0,1
2023-01-01 00:00:02,0.0,0.571429,0.256914,0.50002,0.5002,0.0,0.0,0.33599,0.315382,0.130919,0.116409,0.409741,0.246024,0.14197,0.112819,0.550195,0.406698,0.0,0.0,2
2023-01-01 00:00:03,0.0,0.571429,0.268273,0.50003,0.5003,0.0,0.0,0.33599,0.315382,0.130919,0.116409,0.409758,0.246024,0.14197,0.112819,0.550195,0.40671,0.0,0.0,3
2023-01-01 00:00:04,0.0,0.571429,0.277953,0.50004,0.5004,0.0,0.0,0.33599,0.315382,0.130919,0.116409,0.409782,0.246024,0.14197,0.112819,0.550197,0.406726,0.0,0.0,4


In [7]:
cats_df = cats_df.to_pandas()

In [8]:
potential_causes = metadata['root_cause'].unique().to_list()

In [9]:
cats_df=cats_df.set_index('timestamp')
cats_df = cats_df.drop(['y','category','entity_id'],axis=1)
cats_df.head()


,aimp,amud,arnd,asin1,asin2,adbr,adfl,bed1,bed2,bfo1,bfo2,bso1,bso2,bso3,ced1,cfo1,cso1
timestamp,,,,,,,,,,,,,,,,,
2023-01-01 00:00:00,0.0,0.571429,0.250000,0.50000,0.5000,0.0,0.0,0.33599,0.315382,0.130919,0.116409,0.409726,0.246024,0.14197,0.112819,0.550194,0.406688
2023-01-01 00:00:01,0.0,0.571429,0.252001,0.50001,0.5001,0.0,0.0,0.33599,0.315382,0.130919,0.116409,0.409730,0.246024,0.14197,0.112819,0.550194,0.406691
2023-01-01 00:00:02,0.0,0.571429,0.256914,0.50002,0.5002,0.0,0.0,0.33599,0.315382,0.130919,0.116409,0.409741,0.246024,0.14197,0.112819,0.550195,0.406698
2023-01-01 00:00:03,0.0,0.571429,0.268273,0.50003,0.5003,0.0,0.0,0.33599,0.315382,0.130919,0.116409,0.409758,0.246024,0.14197,0.112819,0.550195,0.406710
2023-01-01 00:00:04,0.0,0.571429,0.277953,0.50004,0.5004,0.0,0.0,0.33599,0.315382,0.130919,0.116409,0.409782,0.246024,0.14197,0.112819,0.550197,0.406726


In [10]:
train_df = cats_df[0:1000000]
test_df = cats_df[1000000:]

In [11]:
train_df.head()

,aimp,amud,arnd,asin1,asin2,adbr,adfl,bed1,bed2,bfo1,bfo2,bso1,bso2,bso3,ced1,cfo1,cso1
timestamp,,,,,,,,,,,,,,,,,
2023-01-01 00:00:00,0.0,0.571429,0.250000,0.50000,0.5000,0.0,0.0,0.33599,0.315382,0.130919,0.116409,0.409726,0.246024,0.14197,0.112819,0.550194,0.406688
2023-01-01 00:00:01,0.0,0.571429,0.252001,0.50001,0.5001,0.0,0.0,0.33599,0.315382,0.130919,0.116409,0.409730,0.246024,0.14197,0.112819,0.550194,0.406691
2023-01-01 00:00:02,0.0,0.571429,0.256914,0.50002,0.5002,0.0,0.0,0.33599,0.315382,0.130919,0.116409,0.409741,0.246024,0.14197,0.112819,0.550195,0.406698
2023-01-01 00:00:03,0.0,0.571429,0.268273,0.50003,0.5003,0.0,0.0,0.33599,0.315382,0.130919,0.116409,0.409758,0.246024,0.14197,0.112819,0.550195,0.406710
2023-01-01 00:00:04,0.0,0.571429,0.277953,0.50004,0.5004,0.0,0.0,0.33599,0.315382,0.130919,0.116409,0.409782,0.246024,0.14197,0.112819,0.550197,0.406726


In [12]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
#device = torch.device("cpu")


In [13]:
device

device(type='cuda')

In [14]:
train_df.columns

Index(['aimp', 'amud', 'arnd', 'asin1', 'asin2', 'adbr', 'adfl', 'bed1',
       'bed2', 'bfo1', 'bfo2', 'bso1', 'bso2', 'bso3', 'ced1', 'cfo1', 'cso1'],
      dtype='object')

In [15]:
tkgnc_data = TKGNGCDataProcessor(train_df,device,num_timestamps=20, lags=1)

In [16]:
pretrained_tkg = PretrainedTKGEmbeddingWithTimestamps(
    num_entities=int(tkgnc_data.entity_indices.max().item()+1),
    num_relations=int(tkgnc_data.relation_indices.max().item()+1),
    embedding_dim=17,
    num_timestamps=tkgnc_data.num_timestamps,
).to(device)

In [17]:
quads = (
    tkgnc_data.entity_indices[:-1],  # Head entities
    tkgnc_data.relation_indices,  # Relations
    tkgnc_data.entity_indices[1:],  # Tail entities (shifted example)
    tkgnc_data.timestamp_indices[:-1],  # Timestamps
)


In [18]:
#pretrained_tkg.pretrain(quads, learning_rate=0.01, epochs=1000)
#torch.save(pretrained_tkg.state_dict(), 'pretrained_tkg')
pretrained_tkg.load_state_dict(torch.load('pretrained_tkg'))

<All keys matched successfully>

In [19]:
from CARAT.data_processing import *
from CARAT.model import *
from CARAT.causal_inference import *

Using device: cuda


C:\Users\jlowh\anaconda3\envs\torch\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [20]:
try:
    train_df = train_df.drop('time',axis=1)
except:
    None
try:
    test_df = test_df.drop('time',axis=1)
except:
    None

In [21]:
cols = list(train_df.columns)
non_causal_columns = list(set(cols).difference(set(potential_causes)))
causal_indices = [train_df.columns.get_loc(col) for col in potential_causes]
non_causal_indices = [train_df.columns.get_loc(col) for col in non_causal_columns]

num_nodes = len(train_df.columns)

new_metadata = []
A0 = get_adjacency(cols,causal_indices,non_causal_indices,num_nodes)    
BATCH_SIZE = 64

model = CausalGraphVAE(input_dim=num_nodes, hidden_dim=128,
                   latent_dim=16, 
                   num_nodes=num_nodes, 
                   embed_dim=17,
                   prior_adj_matrix=A0.to(torch.float))
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

In [22]:
non_causal_columns

['cfo1',
 'aimp',
 'adfl',
 'amud',
 'ced1',
 'adbr',
 'asin1',
 'cso1',
 'arnd',
 'asin2']

In [ ]:
from utils.utils import set_seed
set_seed()
n_correct = 0
method2_correct=0
total_checked = 0
for i, row in enumerate(cats_rows_list):
    total_checked +=1 
    print('Model: '+ str(i))
    anomaly = eval(row['affected'])[0]
    print('Anomaly: ' + anomaly)
    anomaly_time = datetime.strptime(row['start_time'],"%Y-%m-%d %H:%M:%S")
    #start_time = datetime.strptime(row['start_time'],"%Y-%m-%d %H:%M:%S")
    end_time = datetime.strptime(row['end_time'],"%Y-%m-%d %H:%M:%S")
    root_cause = row['root_cause']
    mod_df = test_df[(test_df.index>= anomaly_time) & (test_df.index<= end_time)]
    mod_df = mod_df[['aimp', 'amud', 'arnd', 'asin1', 'asin2', 'adbr', 'adfl', 'bed1',
       'bed2', 'bfo1', 'bfo2', 'bso1', 'bso2', 'bso3', 'ced1', 'cfo1', 'cso1']]
    num_nodes = len(mod_df.columns)

    start_len = mod_df.shape[0]
    if start_len >1000:
        start_len = 1000

    start_time = anomaly_time- timedelta(seconds=start_len)

    normal_df = test_df[(test_df.index>= start_time) & (test_df.index< anomaly_time)]

    sample_data = TKGNGCDataProcessor(mod_df,device,num_timestamps=20, lags=1)
    model_data = create_granger_gat_data(pretrained_tkg,sample_data)
    model_data.retrain_tkg()
    features = model_data.time_series_data
    model_name = 'model_category_'+str(row['category'])
    """model = CausalGraphVAE(input_dim=num_nodes, hidden_dim=128,
                   latent_dim=16, 
                   num_nodes=num_nodes, 
                   embed_dim=17,
                   prior_adj_matrix=A0.to(torch.float))"""
    model = CausalGraphVAE(input_dim=num_nodes, hidden_dim=128,
                   latent_dim=16, 
                   num_nodes=num_nodes, 
                   embed_dim=17,
                   prior_adj_matrix=A0.to(torch.float))
    optimizer = torch.optim.Adam(model.parameters(), lr=0.01)
    data = create_lagged_features(features, 3, pad_value=0)
    ee = create_lagged_features(model_data.entity_emb, 3, pad_value=0)
    tt = create_lagged_features(model_data.timestamp_emb, 3, pad_value=0)
        
    dataset = TimeSeriesDataset(data, ee, tt)
    dataloader = DataLoader(dataset,batch_size = BATCH_SIZE, shuffle=False)
    loss = train_causal_vae(model, optimizer, dataloader, A0, num_epochs=2000)
    print('Evaluating model')
    model.eval()
    non_causal_mask = torch.tensor(A0 != 0, dtype=torch.float64, device=device)
    
    with torch.no_grad():
        _, _, _, W, A = model(data, ee, tt, num_nodes=num_nodes)
        learned_adj_matrix = W + A  # Combine intra- and inter-slice relationships
    
        # Ensure non-causal edges are explicitly zeroed out
        non_causal_mask = torch.tensor(A0 != 0, dtype=torch.float64, device=device)
        learned_adj_matrix = learned_adj_matrix * non_causal_mask  # Apply mask
    
        # Apply a threshold to remove weak edges
        threshold = torch.mean(learned_adj_matrix).item()
        final_W = (learned_adj_matrix.abs() > threshold).float() * learned_adj_matrix
    
        # Clip values to ensure they remain in [0, 1]
        causal_graph = torch.clamp(final_W, 0, 1).cpu().numpy()  # 🚀 Fixes >1 values
    
    print('Beginning inference phase')
    adj_data = pd.DataFrame(causal_graph,index=cols,columns=cols)
    candidates = adj_data[anomaly].sort_values(ascending=False)
    candidates = candidates[candidates.index.isin( potential_causes)]

    ex_cols = [x for x in non_causal_columns if x != anomaly]
    #A0 = learned_adj_matrix
    #try:
    print('Causal engine executing')    
    
    ci = CausalInference(causal_graph, mod_df, normal_df,excluded_nodes=ex_cols)
    cp = CausalPipeline(ci)
    results = cp.run_pipeline(anomaly)


    
    if results is not None:
        sorted_counterfactuals_list = sorted(
            [(key, value['difference_due_to_intervention'])
             for key, value in results['counterfactual_results'].items()],
            key=lambda x: x[1],  # Sort by difference_due_to_intervention
            reverse=True  # Descending order
        )
        # Select the top 3 counterfactuals as a list of tuples
        top_3_counterfactuals_list = sorted_counterfactuals_list[:3]
        print("Final Results:", results)
        if len(top_3_counterfactuals_list) >= 3:
            potential_cause1 = top_3_counterfactuals_list[0][0]
            potential_cause2 = top_3_counterfactuals_list[1][0]
            potential_cause3 = top_3_counterfactuals_list[2][0]
    
        elif len(top_3_counterfactuals_list) == 2:
            potential_cause1 = top_3_counterfactuals_list[0][0]
            potential_cause2 = top_3_counterfactuals_list[1][0]
            for cand in candidates.index:
                if cand not in [potential_cause1,potential_cause2]:
                    potential_cause3 = cand
                    break
        elif len(top_3_counterfactuals_list) == 1:
            potential_cause1 = top_3_counterfactuals_list[0][0]
            for cand in candidates.index:
                if cand not in [potential_cause1]:
                    potential_cause2 = cand
                    break
            for cand in candidates.index:
                if cand not in [potential_cause1,potential_cause2]:
                    potential_cause3 = cand
                    break
        else:
            potential_cause1 = candidates.index[0]
            potential_cause2 = candidates.index[1]
            potential_cause3 = candidates.index[2]
    else:
        potential_cause1 = candidates.index[0]
        potential_cause2 = candidates.index[1]
        potential_cause3 = candidates.index[2]

   
    if root_cause in [potential_cause1,potential_cause2,potential_cause3]:
        n_correct+=1

    if root_cause == potential_cause1:
        row['cause_1'] = 1
    if root_cause == potential_cause2:
        row['cause_2'] = 1
    if root_cause == potential_cause3:
        row['cause_3'] = 1
    new_metadata.append(row)
    if root_cause in [potential_cause1 , potential_cause2 , potential_cause3]:
        print('ROOT CAUSE FOUND!')
    print(root_cause + '-->' + potential_cause1 + ' | ' + potential_cause2 + ' | ' + potential_cause3)

    print("Model Accuracy Method 1: " + str(round( (n_correct/total_checked)*100,2)) + '%' )
    
    print('--------------------------------------------------------------------------------------------------------------------------------')




Model: 0
Anomaly: cfo1
Epoch 0, Loss: 0.0011
Epoch 0: Average Loss = 2995.8868, Best Loss = 2995.8868
Epoch 100: Average Loss = 109.6414, Best Loss = 109.6414
Epoch 200: Average Loss = 53.4233, Best Loss = 53.4233
Epoch 300: Average Loss = 40.7439, Best Loss = 40.7439
Early stopping triggered after 379 epochs. Restoring best model.
Evaluating model
Beginning inference phase
Causal engine executing
🔍 Checking for cycles using Tarjan’s SCC Algorithm...
✅ No cycles detected! Graph is now a DAG.


Fitting causal mechanism of node cso1: 100%|██████████| 17/17 [00:01<00:00, 16.18it/s]


Final Results: {'target_variable': 'cfo1', 'top_causes': ['bso2', 'bed2', 'bso3', 'bed1', 'bso1', 'bfo2', 'bfo1'], 'estimated_effects': {'bso2': 0.0002273335328888959, 'bed2': -0.0004442441982518419, 'bso3': -0.0005654255834364741, 'bed1': -0.0006689760135176193, 'bso1': -0.0034701999431372976, 'bfo2': -0.006797967386668713, 'bfo1': -0.04280491497640487}, 'observed_value': 0.47446008281596735, 'counterfactual_results': {'bso2': {'counterfactual_value': 0.9491412969694277, 'difference_due_to_intervention': 0.4746812141534604}, 'bed2': {'counterfactual_value': 0.9488109993826195, 'difference_due_to_intervention': 0.4743509165666521}, 'bso3': {'counterfactual_value': 0.9493835135328947, 'difference_due_to_intervention': 0.4749234307169274}, 'bed1': {'counterfactual_value': 0.9473174706111205, 'difference_due_to_intervention': 0.47285738779515313}, 'bso1': {'counterfactual_value': 0.9477545738545473, 'difference_due_to_intervention': 0.47329449103857996}, 'bfo2': {'counterfactual_value': 0

Fitting causal mechanism of node cso1: 100%|██████████| 17/17 [00:01<00:00,  9.09it/s]


Final Results: {'target_variable': 'cfo1', 'top_causes': ['bso3', 'bso1', 'bed2', 'bed1', 'bso2', 'bfo2', 'bfo1'], 'estimated_effects': {'bso3': 0.004407722847816653, 'bso1': 0.002103112852526845, 'bed2': 0.0013642848940804697, 'bed1': -0.0008882040186324325, 'bso2': -0.0009840476315635671, 'bfo2': -0.005963770833314563, 'bfo1': -0.05865250539709549}, 'observed_value': 0.5275978435022561, 'counterfactual_results': {'bso3': {'counterfactual_value': 1.0590467608747225, 'difference_due_to_intervention': 0.5314489173724664}, 'bso1': {'counterfactual_value': 1.0552543100485918, 'difference_due_to_intervention': 0.5276564665463357}, 'bed2': {'counterfactual_value': 1.0561000623507077, 'difference_due_to_intervention': 0.5285022188484516}, 'bed1': {'counterfactual_value': 1.054914757071011, 'difference_due_to_intervention': 0.5273169135687548}, 'bso2': {'counterfactual_value': 1.0552108724814993, 'difference_due_to_intervention': 0.5276130289792431}, 'bfo2': {'counterfactual_value': 1.0534264

...

Final Results: {'target_variable': 'cfo1', 'top_causes': ['bed1'], 'estimated_effects': {'bed1': -5.178467215252812}, 'observed_value': 0.5619917985908419, 'counterfactual_results': {'bed1': {'counterfactual_value': -6.969813437568655, 'difference_due_to_intervention': -7.531805236159498}}}
ROOT CAUSE FOUND!
bfo1-->bed1 | bfo1 | bso3
Model Accuracy Method 1: 47.3%
--------------------------------------------------------------------------------------------------------------------------------
Model: 74
Anomaly: cso1
Epoch 0, Loss: 0.0001
Epoch 0: Average Loss = 8071.4156, Best Loss = 8071.4156
Epoch 100: Average Loss = 139.3536, Best Loss = 139.3536
Epoch 200: Average Loss = 60.3430, Best Loss = 60.3430
Epoch 300: Average Loss = 43.0479, Best Loss = 43.0479
Epoch 400: Average Loss = 36.6861, Best Loss = 36.6861
Early stopping triggered after 454 epochs. Restoring best model.
Evaluating model
Beginning inference phase
Causal engine executing
🔍 Checking for cycles using Tarjan’s SCC Algori

Fitting causal mechanism of node cso1: 100%|██████████| 17/17 [00:00<00:00, 20.17it/s]


Final Results: {'target_variable': 'cso1', 'top_causes': ['bfo2', 'bso3', 'bso1', 'bfo1', 'bso2', 'bed2', 'bed1'], 'estimated_effects': {'bfo2': 0.023704195124923055, 'bso3': 0.0011691804990668908, 'bso1': 0.0010775624877111256, 'bfo1': 0.0010539884392418153, 'bso2': -0.0005009036494967312, 'bed2': -0.0012421079688959002, 'bed1': -0.006345586572310935}, 'observed_value': 0.4857333373082345, 'counterfactual_results': {'bfo2': {'counterfactual_value': 0.9795808054475129, 'difference_due_to_intervention': 0.49384746813927843}, 'bso3': {'counterfactual_value': 0.9721961143195803, 'difference_due_to_intervention': 0.4864627770113458}, 'bso1': {'counterfactual_value': 0.9710148789126963, 'difference_due_to_intervention': 0.48528154160446185}, 'bfo1': {'counterfactual_value': 0.9724842266589082, 'difference_due_to_intervention': 0.48675088935067373}, 'bso2': {'counterfactual_value': 0.9707537615287782, 'difference_due_to_intervention': 0.48502042422054376}, 'bed2': {'counterfactual_value': 0.

Fitting causal mechanism of node cso1: 100%|██████████| 17/17 [00:00<00:00, 26.46it/s]


Final Results: {'target_variable': 'cfo1', 'top_causes': ['bed2', 'bso3', 'bso2', 'bso1', 'bed1', 'bfo2', 'bfo1'], 'estimated_effects': {'bed2': 0.0007179792119375028, 'bso3': 0.0006693140132579822, 'bso2': 6.17578754554815e-05, 'bso1': -0.000715219034444381, 'bed1': -0.0011814804237231247, 'bfo2': -0.012750917874762457, 'bfo1': -0.03561261914768277}, 'observed_value': 0.5492647997424204, 'counterfactual_results': {'bed2': {'counterfactual_value': 1.0987822825178188, 'difference_due_to_intervention': 0.5495174827753985}, 'bso3': {'counterfactual_value': 1.0994580315113676, 'difference_due_to_intervention': 0.5501932317689472}, 'bso2': {'counterfactual_value': 1.0986882424123687, 'difference_due_to_intervention': 0.5494234426699484}, 'bso1': {'counterfactual_value': 1.0988887041599917, 'difference_due_to_intervention': 0.5496239044175714}, 'bed1': {'counterfactual_value': 1.0972722440492237, 'difference_due_to_intervention': 0.5480074443068034}, 'bfo2': {'counterfactual_value': 1.089838

In [24]:
if len(top_3_counterfactuals_list) >= 3:
    potential_cause1 = top_3_counterfactuals_list[0][0]
    potential_cause2 = top_3_counterfactuals_list[1][0]
    potential_cause3 = top_3_counterfactuals_list[2][0]

elif len(top_3_counterfactuals_list) == 2:
    potential_cause1 = top_3_counterfactuals_list[0][0]
    potential_cause2 = top_3_counterfactuals_list[1][0]
    for cand in candidates.index:
        if cand not in [potential_cause1,potential_cause2]:
            potential_cause3 = cand
            break
elif len(top_3_counterfactuals_list) == 1:
    potential_cause1 = top_3_counterfactuals_list[0][0]
    for cand in candidates.index:
        if cand not in [potential_cause1]:
            potential_cause2 = cand
            break
    for cand in candidates.index:
        if cand not in [potential_cause1,potential_cause2]:
            potential_cause3 = cand
            break
else:
    potential_cause1 = candidates.index[0]
    potential_cause2 = candidates.index[1]
    potential_cause3 = candidates.index[2]

In [25]:
potential_cause1

'bso3'

In [26]:
top_3_counterfactuals_list

[('bso3', 0.4749234307169274),
 ('bso2', 0.4746812141534604),
 ('bed2', 0.4743509165666521)]

In [32]:
if results is not None:
    top_causes = sorted(results['estimated_effects'].items(), key=lambda item: np.abs(item[1]), reverse=True)
    print("Final Results:", results)
    if len(top_causes) >= 3:
        potential_cause1 = top_causes[0][0]
        potential_cause2 = top_causes[1][0]
        potential_cause3 = top_causes[2][0]

    elif len(top_causes) == 2:
        potential_cause1 = top_causes[0][0]
        potential_cause2 = top_causes[1][0]
        for cand in candidates.index:
            if cand not in [potential_cause1,potential_cause2]:
                potential_cause3 = cand
                break
    elif len(top_causes) == 1:
        potential_cause1 = top_causes[0][0]
        for cand in candidates.index:
            if cand not in [potential_cause1]:
                potential_cause2 = cand
                break
        for cand in candidates.index:
            if cand not in [potential_cause1,potential_cause2]:
                potential_cause3 = cand
                break
    else:
        potential_cause1 = candidates.index[0]
        potential_cause2 = candidates.index[1]
        potential_cause3 = candidates.index[2]
else:
    potential_cause1 = candidates.index[0]
    potential_cause2 = candidates.index[1]
    potential_cause3 = candidates.index[2]

In [33]:
potential_cause1

'bfo1'

In [34]:
sorted_counterfactuals_list = sorted(
    [(key, value['difference_due_to_intervention'])
     for key, value in results['counterfactual_results'].items()],
    key=lambda x: x[1],  # Sort by difference_due_to_intervention
    reverse=True  # Descending order
)

# Select the top 3 counterfactuals as a list of tuples
top_3_counterfactuals_list = sorted_counterfactuals_list[:3]

'bso3'

In [35]:
potential_cause3

'bso2'

In [36]:
root_cause

'bfo1'

In [62]:
for cand in candidates.index:
    if cand not in [potential_cause1,potential_cause2]:
        potential_cause3 = cand
        break

In [63]:
cand

'bfo1'

In [24]:
from dowhy import gcm
import networkx as nx
import numpy as np
import pandas as pd

class CausalInference:
    def __init__(self, adj_matrix, new_data, old_data, excluded_nodes=None):
        self.adj_matrix = adj_matrix
        np.fill_diagonal(self.adj_matrix, 0)
        self.new_data = new_data
        self.old_data = old_data
        self.cols = list(new_data.columns)
        self.excluded_nodes = set(excluded_nodes) if excluded_nodes else set()

        # Build initial graph
        self.causal_graph = nx.DiGraph(self.adj_matrix)
        self.remove_excluded_nodes()
        self.ensure_dag()
        self.build_causal_model()

    def remove_excluded_nodes(self):
        """Remove explicitly excluded nodes from the causal graph."""
        for node in list(self.causal_graph.nodes):
            if node in self.excluded_nodes:
                self.causal_graph.remove_node(node)

    def ensure_dag(self):
        """
        Fully removes all cycles using Tarjan's SCC Algorithm.
        It continues removing edges until no cycles remain.
        """
        print("🔍 Checking for cycles using Tarjan’s SCC Algorithm...")

        while True:
            sccs = list(nx.strongly_connected_components(self.causal_graph))
            cycles = [scc for scc in sccs if len(scc) > 1]  # Only keep actual cycles

            if not cycles:
                print("✅ No cycles detected! Graph is now a DAG.")
                return

            #print(f"⚠️ Cycles detected: {cycles}. Removing edges...")

            for cycle in cycles:
                cycle_edges = [(a, b) for a in cycle for b in self.causal_graph.neighbors(a) if b in cycle]

                # Remove the edge that has the highest out-degree (most influential)
                edge_to_remove = max(cycle_edges, key=lambda e: self.causal_graph.out_degree(e[0]))
                self.causal_graph.remove_edge(*edge_to_remove)

                #print(f"🛑 Removed edge {edge_to_remove} to break cycle.")

    def build_causal_model(self):
        """Constructs the probabilistic causal model using DoWhy."""
        mapping = {i: col for i, col in enumerate(self.cols)}
        self.causal_graph = nx.relabel_nodes(self.causal_graph, mapping)
        self.causal_model_prob = gcm.ProbabilisticCausalModel(self.causal_graph)
        gcm.auto.assign_causal_mechanisms(self.causal_model_prob, self.new_data, override_models=True)
        gcm.fit(self.causal_model_prob, self.new_data)

    def infer_causal_effect(self, cause, effect):
        """Handles standard causal effect inference."""
        return gcm.average_causal_effect(
            self.causal_model_prob, effect,
            interventions_alternative={cause: lambda x: 1},
            interventions_reference={cause: lambda x: 0},
            observed_data=self.new_data
        )

    def infer_counterfactual(self, interventions, num_runs=10):
        """
        Generates counterfactual samples for given interventions.
        
        - **Key Fix:** Resets noise data before each run to prevent identical counterfactuals.
        """
        intervention_dict = {var: (lambda x: value) for var, value in interventions.items()}
        
        # Run counterfactual simulations multiple times
        all_results = []
        for _ in range(num_runs):
            # Deep copy noise data to introduce variability
            noise_data_sampled = self.new_data.sample(frac=1, replace=True).reset_index(drop=True)

            cf_result = gcm.counterfactual_samples(
                self.causal_model_prob,
                intervention_dict,
                noise_data=noise_data_sampled
            )
            all_results.append(cf_result)

        # Average the results across multiple runs
        avg_results = pd.concat(all_results).groupby(level=0).mean()
        return avg_results


    def rank_causal_factors(self, target):
        """Ranks the top causal factors influencing the target variable."""
        causal_effects = {
            cause: self.infer_causal_effect(cause, target)
            for cause in self.causal_graph.predecessors(target)
            if cause not in self.excluded_nodes
        }
        return sorted(causal_effects.items(), key=lambda item: item[1], reverse=True)

    def draw_causal_dag(self):
        """Visualizes the causal DAG."""
        nx.draw(self.causal_graph, with_labels=True, node_size=800, node_color="skyblue",
                font_size=10, font_weight="bold", arrowsize=20)

class CausalPipeline:
    def __init__(self, causal_inference):
        self.causal_inference = causal_inference

    def run_pipeline(self, target_variable, num_runs=10):
        """
        Runs separate counterfactual analyses for each top causal factor.
        """
        # 1️⃣ Rank the most important causal factors
        ranked_causes = self.causal_inference.rank_causal_factors(target_variable)

        if not ranked_causes:
            print(f"⚠️ No causal factors found for {target_variable}. Exiting pipeline.")
            return None

        # Take the top 3 causes (or fewer if not available)
        top_causes = ranked_causes[:3]

        # 2️⃣ Estimate the direct causal effects for the top causes
        estimated_effects = {
            cause: self.causal_inference.infer_causal_effect(cause, target_variable)
            for cause, _ in top_causes
        }

        # Filter out None values (due to instantaneous effects)
        estimated_effects = {k: v for k, v in estimated_effects.items() if v is not None}

        if not estimated_effects:
            print(f"⚠️ No valid causal effects found for {target_variable}. Exiting pipeline.")
            return None

        # 3️⃣ Generate counterfactual outcomes for EACH cause separately
        counterfactual_results = {}
        observed_value = self.causal_inference.new_data[target_variable].mean()

        for cause in estimated_effects.keys():
            intervention = {cause: 1}  # Apply intervention only on this cause

            # Generate counterfactual results with multiple runs for variability
            cf_result = self.causal_inference.infer_counterfactual(intervention, num_runs=num_runs)

            if target_variable in cf_result:
                counterfactual_value = np.mean(cf_result[target_variable])
                counterfactual_results[cause] = {
                    "counterfactual_value": counterfactual_value,
                    "difference_due_to_intervention": counterfactual_value - observed_value
                }
            else:
                print(f"⚠️ Counterfactual results missing for {target_variable} under intervention {cause}.")
                counterfactual_results[cause] = None

        # 4️⃣ Store structured results
        results = {
            "target_variable": target_variable,
            "top_causes": list(estimated_effects.keys()),
            "estimated_effects": estimated_effects,
            "observed_value": observed_value,
            "counterfactual_results": counterfactual_results
        }

        return results


In [25]:
ex_cols = [x for x in non_causal_columns if x != anomaly]

In [26]:
ex_cols

['asin1', 'cso1', 'aimp', 'adfl', 'ced1', 'arnd', 'amud', 'asin2', 'adbr']

In [51]:
non_causal_mask = torch.tensor(A0 != 0, dtype=torch.float64, device=device)

with torch.no_grad():
    _, _, _, W, A = model(data, ee, tt, num_nodes=num_nodes)
    learned_adj_matrix = W + A  # Combine intra- and inter-slice relationships

    # Ensure non-causal edges are explicitly zeroed out
    non_causal_mask = torch.tensor(A0 != 0, dtype=torch.float64, device=device)
    learned_adj_matrix = learned_adj_matrix * non_causal_mask  # Apply mask

    # Apply a threshold to remove weak edges
    threshold = torch.mean(learned_adj_matrix).item()
    final_W = (learned_adj_matrix.abs() > threshold).float() * learned_adj_matrix

    # Clip values to ensure they remain in [0, 1]
    causal_graph = torch.clamp(final_W, 0, 1).cpu().numpy()  # 🚀 Fixes >1 values


In [52]:
causal_graph

array([[0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        ],
       [0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        ],
       [0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        ],
       [0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        ],
       [0.        , 0.        , 0.        , 0.      

In [53]:
ci = CausalInference(causal_graph, mod_df, normal_df,excluded_nodes=ex_cols)
cp = CausalPipeline(ci)
cp.run_pipeline(anomaly)

🔍 Checking for cycles using Tarjan’s SCC Algorithm...
✅ No cycles detected! Graph is now a DAG.


Fitting causal mechanism of node cso1: 100%|██████████| 17/17 [00:01<00:00, 16.10it/s]


{'target_variable': 'cfo1',
 'top_causes': ['bso2', 'bed2', 'bso3'],
 'estimated_effects': {'bso2': 0.0002273335328888959,
  'bed2': -0.0004442441982518419,
  'bso3': -0.0005654255834364741},
 'observed_value': 0.47446008281596735,
 'counterfactual_results': {'bso2': {'counterfactual_value': 0.9488493572053863,
   'difference_due_to_intervention': 0.47438927438941897},
  'bed2': {'counterfactual_value': 0.9487550774644229,
   'difference_due_to_intervention': 0.47429499464845554},
  'bso3': {'counterfactual_value': 0.9489003435853061,
   'difference_due_to_intervention': 0.4744402607693387}}}

In [ ]:
import copy

class CausalInference:
    def __init__(self, adj_matrix, new_data, old_data, excluded_nodes=None):
        self.adj_matrix = adj_matrix
        np.fill_diagonal(self.adj_matrix, 0)
        self.new_data = new_data
        self.old_data = old_data
        self.cols = list(new_data.columns)
        self.excluded_nodes = set(excluded_nodes) if excluded_nodes else set()

        # Build initial graph
        self.causal_graph = nx.DiGraph(self.adj_matrix)
        self.remove_excluded_nodes()
        self.ensure_dag()
        self.build_causal_model()

    def infer_counterfactual(self, interventions, num_runs=10):
        """
        Generates counterfactual samples for given interventions.
        
        - **Key Fix:** Resets noise data before each run to prevent identical counterfactuals.
        """
        intervention_dict = {var: (lambda x: value) for var, value in interventions.items()}
        
        # Run counterfactual simulations multiple times
        all_results = []
        for _ in range(num_runs):
            # Deep copy noise data to introduce variability
            noise_data_sampled = self.new_data.sample(frac=1, replace=True).reset_index(drop=True)

            cf_result = gcm.counterfactual_samples(
                self.causal_model_prob,
                intervention_dict,
                noise_data=noise_data_sampled
            )
            all_results.append(cf_result)

        # Average the results across multiple runs
        avg_results = pd.concat(all_results).groupby(level=0).mean()
        return avg_results


In [44]:
cp = CausalPipeline(ci)

In [46]:
results = cp.run_pipeline(anomaly)

In [47]:
results

{'target_variable': 'cfo1',
 'top_causes': [('bso2', 0.0001834267798319078),
  ('bso1', 1.1901378589307399e-06),
  ('bso3', -0.00030725587418706235)],
 'estimated_effects': {'bso2': 0.0001958691214440922,
  'bso1': -5.541752753857043e-06,
  'bso3': -0.000312007507375478},
 'observed_value': 0.47446008281596735,
 'counterfactual_value': 0.923078182862275,
 'difference_due_to_intervention': 0.4486181000463077}

In [49]:
results['top_causes']

[('bso2', 0.0001834267798319078),
 ('bso1', 1.1901378589307399e-06),
 ('bso3', -0.00030725587418706235)]

In [52]:
top_causes = sorted(results['top_causes'], key=lambda item: np.abs(item[1]), reverse=True)

In [27]:
adj_data = pd.DataFrame(causal_graph,index=cols,columns=cols)
candidates = adj_data[anomaly].sort_values(ascending=False)

In [26]:
model.eval()
with torch.no_grad():
    _, _, _, learned_adj_matrix = model(data,ee,tt, num_nodes=num_nodes)
    # small threshold
    #threshold = torch.mean(learned_adj_matrix).item()
    #final_W = (learned_adj_matrix.abs() > threshold).float() * learned_adj_matrix
    causal_graph = torch.sigmoid(learned_adj_matrix).cpu().numpy()

In [37]:
adj_data = pd.DataFrame(causal_graph,index=cols,columns=cols)
candidates = adj_data[anomaly].sort_values(ascending=False)
candidates

bfo1     0.608825
bso3     0.598015
bed1     0.583635
bso2     0.582863
bfo2     0.578179
bso1     0.576747
bed2     0.569972
asin1    0.529338
amud     0.522121
adbr     0.519892
arnd     0.517744
asin2    0.515766
adfl     0.511354
aimp     0.506731
ced1     0.504005
cso1     0.503527
cfo1     0.500920
Name: cfo1, dtype: float64

In [27]:
adj_matrix = causal_graph
np.fill_diagonal(adj_matrix, 0)
new_data = mod_df
old_data = normal_df
cols = list(new_data.columns)
causal_graph = nx.DiGraph(adj_matrix)
#num_cycles = len(list(nx.simple_cycles(causal_graph)))

In [30]:
for non_cause in non_causal_columns:
    causal_graph.remove_edge(anomaly, non_cause)
    

NetworkXError: The edge cfo1-arnd not in graph.

In [ ]:
causal_graph.remove_edge(anomaly, 1)

In [ ]:
non_causal_columns

In [34]:
topological_remove_cycles(causal_graph) 

In [36]:
mapping = {i: col for i, col in enumerate(cols)}
causal_graph = nx.relabel_nodes(causal_graph, mapping)
causal_model_prob = gcm.ProbabilisticCausalModel(causal_graph)
gcm.auto.assign_causal_mechanisms(causal_model_prob, new_data, override_models=True)
gcm.fit(causal_model_prob, new_data)

Fitting causal mechanism of node cso1: 100%|██████████| 17/17 [00:01<00:00, 10.23it/s]


In [78]:
from dowhy import gcm
import networkx as nx
import numpy as np
import pandas as pd

class CausalInference:
    def __init__(self, adj_matrix, new_data, old_data):
        self.adj_matrix = adj_matrix
        np.fill_diagonal(self.adj_matrix, 0)
        self.new_data = new_data
        self.old_data = old_data
        self.cols = list(new_data.columns)
        self.causal_graph = nx.DiGraph(self.adj_matrix)
        #self.num_cycles = len(list(nx.simple_cycles(self.causal_graph)))
        #print('Number of cycles: '+ str(len(list(nx.simple_cycles(self.causal_graph))) ))
        #if len(list(nx.simple_cycles(self.causal_graph))) > 0:
        self.topological_remove_cycles(self.causal_graph) 
        self.build_causal_model()

    def topological_remove_cycles(self, g):
    
            # Taken from: https://stackoverflow.com/questions/78348739/converting-cyclic-digraph-to-acyclic-digraph-dag
            # Dictionary of sets of incoming edges. We want to pick nodes with few of them.
            incoming = {}
            for node in g.nodes:
                incoming[node] = set()
            for node in g.nodes():
                for next_node in g.neighbors(node):
                    incoming[next_node].add(node)
        
            # Sorted set of (count_incoming, -count_outgoing, node) triplets.
            # The first item in the set will have as few incoming nodes as it can, and as many outgoing.
            # In other words, it is a greedy choice for a good node to get rid of cycles.
            todo = SortedSet()
            for node, to_node in incoming.items():
                todo.add((len(to_node), -len(g.adj[node]), node))
        
            # And now let's start removing cycles.
            while 0 < len(todo):
                # Get the best node.
                _, _, node = todo.pop(0)
                to_node = incoming[node]
                for prev_node in to_node:
                    # Each of these edges is in, or comes from, a cycle.
                    if prev_node != node:
                        # Do bookkeeping in todo.
                        len_in = len(incoming[prev_node])
                        len_out = len(g.adj[prev_node])
                        todo.remove((len_in, -len_out, prev_node))
                        todo.add((len_in, -len_out + 1, prev_node))
                    g.remove_edge(prev_node, node)
        
                for next_node in g.neighbors(node):
                    # Do bookkeeping in todo for forgetting about node.
                    len_in = len(incoming[next_node])
                    len_out = len(g.adj[next_node])
                    todo.remove((len_in, -len_out, next_node))
                    todo.add((len_in - 1, -len_out, next_node))
                    incoming[next_node].remove(node)

    def build_causal_model(self):
        mapping = {i: col for i, col in enumerate(self.cols)}
        self.causal_graph = nx.relabel_nodes(self.causal_graph, mapping)
        self.causal_model_prob = gcm.ProbabilisticCausalModel(self.causal_graph)
        gcm.auto.assign_causal_mechanisms(self.causal_model_prob, self.new_data, override_models=True)
        gcm.fit(self.causal_model_prob, self.new_data)

    def infer_causal_effect(self, cause, effect):
        return gcm.average_causal_effect(
            self.causal_model_prob, effect,
            interventions_alternative={cause: lambda x: 1},
            interventions_reference={cause: lambda x: 0},
            observed_data = self.new_data ,
            #num_samples_to_draw=1000
        )

    def infer_counterfactual(self, intervention_var, intervention_value):
        return gcm.counterfactual_samples(
            self.causal_model_prob,
            {intervention_var: lambda x: intervention_value},
            noise_data = self.new_data
          #  num_samples=1000
        )

    def rank_causal_factors(self, target):
        causal_effects = {cause: self.infer_causal_effect(cause, target) for cause in self.causal_graph.predecessors(target)}
        return sorted(causal_effects.items(), key=lambda item: item[1], reverse=True)

    def draw_causal_dag(self):
        nx.draw(self.causal_graph, with_labels=True, node_size=800, node_color="skyblue",
                font_size=10, font_weight="bold", arrowsize=20)


In [79]:
class CausalPipeline:
    def __init__(self, causal_inference):
        self.causal_inference = causal_inference

    def run_pipeline(self, target_variable):
        """
        Automates the process of ranking causal factors, estimating effects,
        generating counterfactuals, and comparing results.
        """
       #print(f"Running Causal Analysis for target: {target_variable}")

        # 1️⃣ Rank the most important causal factors
        ranked_causes = self.causal_inference.rank_causal_factors(target_variable)
        #rint(f"Top causal influences on {target_variable}: {ranked_causes}")

        # 2️⃣ Select top-ranked causes for intervention
        top_cause, max_effect = ranked_causes[0]
       # print(f"Most influential cause: {top_cause} with estimated effect {max_effect}")

        # 3️⃣ Estimate the direct causal effect
        estimated_effect = self.causal_inference.infer_causal_effect(top_cause, target_variable)
        #print(f"Estimated causal effect of {top_cause} on {target_variable}: {estimated_effect}")

        # 4️⃣ Generate counterfactual outcomes
        counterfactual_results = self.causal_inference.infer_counterfactual(top_cause, 1)  # Assume intervention = 1
        #print(f"Counterfactual outcome if {top_cause} was set to 1: {counterfactual_results}")

        # 5️⃣ Compare counterfactuals with observed data
        observed_value = self.causal_inference.new_data[target_variable].mean()
        counterfactual_value = np.mean(counterfactual_results[target_variable])

        #print(f"Observed {target_variable}: {observed_value}")
        #print(f"Counterfactual {target_variable} (if {top_cause} was changed): {counterfactual_value}")

        # 6️⃣ Store structured results
        results = {
            "target_variable": target_variable,
            "top_cause": top_cause,
            "estimated_effect": estimated_effect,
            "observed_value": observed_value,
            "counterfactual_value": counterfactual_value,
            "difference_due_to_intervention": counterfactual_value - observed_value
        }

        return results


In [80]:
causal_inference = CausalInference(causal_graph, mod_df, normal_df)


Fitting causal mechanism of node cso1: 100%|██████████| 17/17 [00:01<00:00, 10.83it/s]


In [81]:
# Initialize Pipeline
pipeline = CausalPipeline(causal_inference)

# Run for a specific target variable
results = pipeline.run_pipeline(anomaly)
print("Final Results:", results)


Running Causal Analysis for target: cfo1
Top causal influences on cfo1: [('arnd', 0.041663062309698795), ('aimp', 0.0014028333316554176), ('amud', 0.0012983340216226602), ('adbr', 0.00019256080588636593), ('bso2', 0.00018679593623593949), ('asin2', 3.637506217085695e-05), ('adfl', 1.7185021351173724e-05), ('bso1', 1.2919063249738905e-06), ('bso3', -0.0003103915477442043), ('bed2', -0.0005711839219293968), ('bed1', -0.0022367851695283547), ('ced1', -0.002429068581546401), ('bfo2', -0.005088972390404178), ('bfo1', -0.005981348709882872), ('asin1', -0.03437310892538448)]
Most influential cause: arnd with estimated effect 0.041663062309698795
Estimated causal effect of arnd on cfo1: 0.04274051755231906
Counterfactual outcome if arnd was set to 1:      aimp      amud  arnd     asin1       asin2  adbr  adfl      bed1  \
0     0.0  0.809524     1  1.950041  173.165366   1.0   1.0  0.748577   
1     0.0  0.809524     1  1.950044  173.167248   1.0   1.0  0.748234   
2     0.0  0.809524     1  1